In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
structure = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "11063126568488847882", 2, "feature")

In [4]:
mat = Matrix{Float64}(LinearAlgebra.I, 3, 3)

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

## Vecchia funzione

In [5]:
function traversal(CTM::Matrix, stack, obj, scene=[])
	for i = 1:length(obj.body)
		if isa(obj.body[i],Matrix)
			CTM = CTM*obj.body[i]
		elseif (isa(obj.body[i],Tuple) || isa(obj.body[i],Array)) && (length(obj.body[i])>=2)
			l = Lar.apply(CTM, obj.body[i])
			push!(scene,l)
		elseif isa(obj.body[i],Lar.Struct)
			push!(stack,CTM)
			Lar.traversal(CTM,stack,obj.body[i],scene)
			CTM = pop!(stack)
		end
	end
	return scene
end

traversal (generic function with 2 methods)

In [6]:
@btime traversal(mat,[],structure,[])

  2.275 μs (20 allocations: 1.33 KiB)


1-element Array{Any,1}:
 ([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [7]:
@benchmark traversal(mat,[],structure,[])

BenchmarkTools.Trial: 
  memory estimate:  1.33 KiB
  allocs estimate:  20
  --------------
  minimum time:     2.289 μs (0.00% GC)
  median time:      5.256 μs (0.00% GC)
  mean time:        6.343 μs (2.46% GC)
  maximum time:     831.333 μs (98.14% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [8]:
@code_warntype traversal(mat,[],structure,[])

Variables
  #self#::Core.Compiler.Const(traversal, false)
  CTM@_2::Array{Float64,2}
  stack::Array{Any,1}
  obj::LinearAlgebraicRepresentation.Struct
  scene::Array{Any,1}
  @_6::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  l::Any
  CTM@_9::Any
  @_10::Bool
  @_11::Any

Body::Array{Any,1}
1 ──       (CTM@_9 = CTM@_2)
│    %2  = Base.getproperty(obj, :body)::Array
│    %3  = Main.length(%2)::Int64
│    %4  = (1:%3)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│          (@_6 = Base.iterate(%4))
│    %6  = (@_6 === nothing)::Bool
│    %7  = Base.not_int(%6)::Bool
└───       goto #16 if not %7
2 ┄─       Core.NewvarNode(:(l))
│    %10 = @_6::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│          (i = Core.getfield(%10, 1))
│    %12 = Core.getfield(%10, 2)::Int64
│    %13 = Base.getproperty(obj, :body)::Array
│    %14 = Base.getindex(%13, i)::Any
│    %15 = (%14 isa Main.Matrix)::Bool
└───       goto #4 if not %15
3 ── %17 = CTM@_9::Any
│    %18 

## Modifiche